In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

import pyarrow as pa
import pyarrow.parquet as pq

import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS
from linearmodels.panel import PanelOLS

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\panel\data.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Categorical, DataFrame, Index, MultiIndex, Panel, Series,


### Overview. 

This file essentially grabs the month by county files from the [Quarterly Census of Employment and Wages](https://www.bls.gov/cew/) files from the BLS and then creates employment measures at the county-level, monthly frequency. A couple of comments about the code:

   - In the funciton ``clean_bls_quarter`` there is a line to be uncommented or not depending upon if I want a dataset with goods employment or total employment. Future enhancements of this notebook should just return one dataframe with both.
    
    
   - It can accomadate the 2016 data (and further back if modified). Currently it just uses the 2017, 2018, and 2019 (which only have Q1 values. See the relese calander when updates will be made.
   
### Step 1

Bring in the trade/tariff data for which we will merge stuff....

In [2]:
cwd = os.getcwd()

trade_data = pq.read_table(cwd + "\\data\\total_trade_data.parquet").to_pandas()

trade_data["time"] = pd.to_datetime(trade_data.time)

trade_data.set_index(["area_fips", "time"],inplace = True)

In [3]:
trade_data.head()

exposure = pd.qcut(trade_data.xs('2018-12-1', level=1).tariff, 4 ,labels = False)

most_exposed = exposure[exposure == 3].index.tolist()

trade_data.loc[most_exposed].xs('2018-12-1', level=1).tariff.mean()

6.4691811550724125

This is ultra-clunky. Should fix in the future. But it takes names (which is how the BLS files are written) and then will map them into a datatime value.

In [4]:
years = [15,16,17,18,19]

empl_time_dict = {}

foo = 0

for xxx in years:
    
    
    year = int("20" + str(xxx))
    
    
    empl_time_dict["year_{0}".format(year)] = {"January Employment":dt.datetime(year,1,1),
                 "February Employment":dt.datetime(year,2,1),
                 "March Employment":dt.datetime(year,3,1),
                 "April Employment":dt.datetime(year,4,1),
                 "May Employment":dt.datetime(year,5,1),
                 "June Employment":dt.datetime(year,6,1),
                 "July Employment":dt.datetime(year,7,1),
                 "August Employment":dt.datetime(year,8,1),
                 "September Employment":dt.datetime(year,9,1),
                 "October Employment":dt.datetime(year,10,1),
                 "November Employment":dt.datetime(year,11,1),
                 "December Employment":dt.datetime(year,12,1),}

In [6]:
empl_time_dict["year_2019"]

{'January Employment': datetime.datetime(2019, 1, 1, 0, 0),
 'February Employment': datetime.datetime(2019, 2, 1, 0, 0),
 'March Employment': datetime.datetime(2019, 3, 1, 0, 0),
 'April Employment': datetime.datetime(2019, 4, 1, 0, 0),
 'May Employment': datetime.datetime(2019, 5, 1, 0, 0),
 'June Employment': datetime.datetime(2019, 6, 1, 0, 0),
 'July Employment': datetime.datetime(2019, 7, 1, 0, 0),
 'August Employment': datetime.datetime(2019, 8, 1, 0, 0),
 'September Employment': datetime.datetime(2019, 9, 1, 0, 0),
 'October Employment': datetime.datetime(2019, 10, 1, 0, 0),
 'November Employment': datetime.datetime(2019, 11, 1, 0, 0),
 'December Employment': datetime.datetime(2019, 12, 1, 0, 0)}

In [5]:


clistQ1 = ['Area\nCode','NAICS','Qtr','January Employment', 'February Employment',
       'March Employment', 'Total Quarterly Wages', 'Average Weekly Wage','Own',"Area Type"]

### Step 2: Download

This downloads the ``.zip`` files for which we can grab the data. They are all in excell format. 

In [36]:
url = "https://data.bls.gov/cew/data/files/2015/xls/2015_all_county_high_level.zip"
# This will read in the annual, single file. It's big, but has all we want...

r = requests.get(url) 

# convert bytes to zip file  
bls_q2015 = zf.ZipFile(io.BytesIO(r.content)) 
bls_q2015.extractall(cwd + "\\bls_files")

In [6]:
url = "https://data.bls.gov/cew/data/files/2016/xls/2016_all_county_high_level.zip"
# This will read in the annual, single file. It's big, but has all we want...

r = requests.get(url) 

# convert bytes to zip file  
bls_q2016 = zf.ZipFile(io.BytesIO(r.content)) 
bls_q2016.extractall(cwd + "\\bls_files")

url = "https://data.bls.gov/cew/data/files/2017/xls/2017_all_county_high_level.zip"
# This will read in the annual, single file. It's big, but has all we want...

r = requests.get(url) 

# convert bytes to zip file  
bls_q2017 = zf.ZipFile(io.BytesIO(r.content)) 
bls_q2017.extractall(cwd + "\\bls_files")

url = "https://data.bls.gov/cew/data/files/2018/xls/2018_all_county_high_level.zip"

r = requests.get(url) 

bls_q2018 = zf.ZipFile(io.BytesIO(r.content)) 
bls_q2018.extractall(cwd + "\\bls_files")

In [7]:
url = "https://data.bls.gov/cew/data/files/2019/xls/2019_all_county_high_level.zip"

r = requests.get(url) 

bls_q2019 = zf.ZipFile(io.BytesIO(r.content)) 
bls_q2019.extractall(cwd + "\\bls_files")

In [8]:
bls_q2019.filelist

[<ZipInfo filename='allhlcn191.xlsx' compress_type=deflate external_attr=0x20 file_size=6839716 compress_size=6715278>]

   ### Step 3: Clean and Shape it
   
   Below is a function that takes in an excell sheet and does what we want to it. Then below we will work through a for loop over all the sheets.

In [68]:
def bls_quarter_cat(df,cat,var_name, time_dict):
    
    # Take only private

    df = df[df["Own"] == 5] 

# Take aggregate

    #df = df[df["NAICS"] == 101] # Take goods producing 
    
    df = df[df["NAICS"] == cat] # Take all employment in all sectors

# Take only counties 
    df = df[df["Area Type"] == "County"] 

    df.rename({"Area\nCode": "GEOFIPS"},axis = 1, inplace = True)

    df["GEOFIPS"] = df["GEOFIPS"].astype(int)

    df.set_index("GEOFIPS", inplace = True)

    df = df.reindex(trade_data.index.get_level_values(0).unique().astype(int).tolist())

    df = df.iloc[:,[13,14,15]].reset_index()
    # This grabs only values we want, i.e. the employment for that quarter. So for example,
    # in Q1, 13 = January, 14 = Febuary, 15 = March. And so forth for Q2...

    df = df.melt("GEOFIPS")

    df.replace(time_dict,inplace = True)

    df.rename({"variable":"time", "value": var_name, "GEOFIPS": "area_fips"}, axis = 1, inplace = True)
    
    df["area_fips"] = df["area_fips"].astype(str)
    
    df.set_index(["area_fips", "time"], inplace = True)
    
    return df

In [67]:
def clean_bls_quarter(excell_sheet, time_dict):

    foo = pd.read_excel(excell_sheet, sheet_name = "US_St_Cn_MSA")
    
    naics_cats = [10,101]
    
    var_name = {10: "emp_all", 101:"emp_gds"}
    
    cat_dict = {}
    
    for xxx in naics_cats:
        
        foo_df = bls_quarter_cat(foo, xxx, var_name[xxx], time_dict)
        
        cat_dict["cat_{0}".format(xxx)] = foo_df
                
    df = cat_dict["cat_10"].merge(cat_dict["cat_101"], left_index = True, right_index = True)

    return df

Then given the function above, work through the file list. 

In [69]:
df = pd.DataFrame([])

############################################################################
root_name = cwd + "\\bls_files\\"

root_name = root_name + "allhlcn15"

quarter = ["1","2","3","4"]

for item in quarter:
    
    file_name = root_name + item + ".xlsx"
    
    df = df.append(clean_bls_quarter(file_name,empl_time_dict["year_2015"]))

############################################################################

root_name = cwd + "\\bls_files\\"

root_name = root_name + "allhlcn16"

quarter = ["1","2","3","4"]

for item in quarter:
    
    file_name = root_name + item + ".xlsx"
    
    df = df.append(clean_bls_quarter(file_name,empl_time_dict["year_2016"]))
    
############################################################################

root_name = cwd + "\\bls_files\\"

root_name = root_name + "allhlcn17"

quarter = ["1","2","3","4"]

for item in quarter:
    
    file_name = root_name + item + ".xlsx"
    
    df = df.append(clean_bls_quarter(file_name,empl_time_dict["year_2017"]))
    
############################################################################  
root_name = cwd + "\\bls_files\\"

root_name = root_name + "allhlcn18"

quarter = ["1","2","3","4"]

for item in quarter:
    
    file_name = root_name + item + ".xlsx"
    
    df = df.append(clean_bls_quarter(file_name,empl_time_dict["year_2018"]))
    
############################################################################  
root_name = cwd + "\\bls_files\\"

root_name = root_name + "allhlcn19"

quarter = ["1"]

for item in quarter:
    
    file_name = root_name + item + ".xlsx"
    
    df = df.append(clean_bls_quarter(file_name,empl_time_dict["year_2019"]))

Then just checksome stuff, reshape, then save for the analysis part. Note how this is working (again clunky), if you want the goods employment, uncomment out that. If you want total employment do the other one.

In [70]:
df.sort_values(["area_fips", "time"], inplace = True)

In [71]:
df.dtypes

emp_all    float64
emp_gds    float64
dtype: object

In [75]:
#df.loc["10003"]

In [76]:
trade_employ = trade_data.merge(df, left_index = True, right_index = True, how = "right")
# This is a place to be mindfull about time period, if we want 
# do left if you just want to conform with the trade data

In [77]:
trade_employ.total_employment.fillna(method='bfill', inplace = True)

trade_employ.tariff.fillna(method='bfill', inplace = True)

In [78]:
file_path = os.getcwd() + "\\data\\trade_employment_2015.parquet"

pq.write_table(pa.Table.from_pandas(trade_employ.reset_index()), file_path)

#file_path = os.getcwd() + "\\data\\trade_employment_all.parquet"

#pq.write_table(pa.Table.from_pandas(trade_employ.reset_index()), file_path)

In [79]:
trade_employ.dtypes

total_exp_pc        float64
china_exp_pc        float64
tariff              float64
emplvl_2017         float64
fips                 object
total_employment    float64
emp_all             float64
emp_gds             float64
dtype: object

In [80]:
trade_employ.loc["10003"]

,total_exp_pc,china_exp_pc,tariff,emplvl_2017,fips,total_employment,emp_all,emp_gds
time,,,,,,,,
2015-01-01,NaN,NaN,0.211322,NaN,NaN,249775.0,244382.0,24054.0
2015-02-01,NaN,NaN,0.211322,NaN,NaN,249775.0,244283.0,24920.0
2015-03-01,NaN,NaN,0.211322,NaN,NaN,249775.0,245084.0,24994.0
2015-04-01,NaN,NaN,0.211322,NaN,NaN,249775.0,247395.0,25741.0
2015-05-01,NaN,NaN,0.211322,NaN,NaN,249775.0,250443.0,26324.0
2015-06-01,NaN,NaN,0.211322,NaN,NaN,249775.0,251350.0,26391.0
2015-07-01,NaN,NaN,0.211322,NaN,NaN,249775.0,252602.0,26631.0
2015-08-01,NaN,NaN,0.211322,NaN,NaN,249775.0,251595.0,26805.0
2015-09-01,NaN,NaN,0.211322,NaN,NaN,249775.0,250168.0,26689.0
